In [27]:
import os
from dotenv import load_dotenv

from mastodon import Mastodon

load_dotenv()

mastodon = Mastodon(
    client_id=os.environ["MASTODON_CLIENT_ID"], 
    client_secret=os.environ["MASTODON_CLIENT_SECRET"],
    api_base_url="https://hachyderm.io",
)
mastodon.log_in("johnturner@me.com", os.environ["MASTODON_PASSWORD"], scopes=["read"])
tl = mastodon.timeline_home()

In [44]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.utilities import TextRequestsWrapper

llm = OpenAI()

prompt = PromptTemplate(input_variables=["toot"], template="""
You are a helpful data entry assistant assigned to classify social media posts as positive, negative, or neutral in tone and subject matter. 
You must only ever output the words positive, negative, or neutral, in all lower case letters.
Never explain your reasoning, only give the rating.

## Examples

# Text: "Brief introduction to dependency management in Go"
# Output: neutral

# Text: "Fucking guns"
# Output: negative

# Text: "This Pharmacist Said Prisoners Wouldn’t Feel Pain During Lethal Injection. Then Some Shook and Gasped for Air."
# Output: negative

# Text: "Some cool news to wake up to: The Kaiju Preservation Society is a 2023 Locus Award finalist, in the category of Science Fiction novel, along with work from Maurice Broaddus, Aliette de Bodard, Mary Robinette Kowal, Emily St. John Mandel, Silvia Moreno-Garcia, Tochi Onyebuchi, Adrian Tchaikovsky, Lavie Tidhar and Alastair Reynolds. That's a pretty great peer group. You can see all the finalists in all the categories at the link below. Congratulations to everyone!"
# Output: positive

## Classify this

# Text: {toot}
# Output: 
""")

chain = LLMChain(llm=llm, prompt=prompt)


In [41]:
from bs4 import BeautifulSoup

def clean(content):
    soup = BeautifulSoup(content)
    return soup.text


In [45]:
for toot in tl:
    content = toot.content
    if not content:
        content = toot.reblog.content
    content = clean(content)
    resp = chain.run(toot=content)
    print(f"Toot: {content}")
    print(f"Rating: {resp}")

Toot: If you like Mastodon and would like more people on here, it might be good to think about how to make it friendlier and more welcoming—instead of ranting about other sites.You sound exactly like the conservatives on Twitter.
Rating: 
neutral
Toot: @morecowbell I just figured out why you gave such a zany and off topic answer when I sent you to my FAQ page.You probably wen to my "To Do" list.I was sending you to these two blog posts:https://terikanefield.com/all-new-doj-investigation-faqs/andhttps://terikanefield.com/frequently-asserted-rage-inducing-simplifications/Just FYI.
Rating: neutral
Toot: Hey, here’s my usual breakdown of Amazon's quarterly business results: https://www.tbray.org/ongoing/When/202x/2023/04/27/Amazon-A1-2023Tl;dr: Amazon retail loses money, but AWS and Advertising emit floods of profit that cover those losses.  Does this sound sane to you?AWS’s growth is slowing but it’s still a damn fine business and should been have spun off from the retail behemoth years a